<a href="https://colab.research.google.com/github/adithyakini/algo_intraday/blob/master/bhavcopy_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install chart_studio

     |████████████████████████████████| 71kB 1.9MB/s 


In [9]:
import mysql.connector
from mysql.connector import Error
from termcolor import colored, cprint
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly.io as pio

######################################################################################
# connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
# pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
# https://pynative.com/python-mysql-database-connection/
symbol='GBPINR'
#enter date of expiry , for ex if its october expiry you want then enter exp_date1='2020-10-01' and exp_date2='2020-11-01'
exp_date1='2020-10-01'
exp_date2='2020-11-01'
bhavcopydate='2020-09-25'
try:
  connection = mysql.connector.connect(host='johnny.heliohost.org',
                                      database='akini_algotrade',
                                      user='akini',
                                      password='Drink7up@home')
  
  db_Info = connection.get_server_info()
  print("\nConnected to MySQL Server - version", db_Info)
  cursor = connection.cursor()
  #cursor.execute("select database();")
  #record = cursor.fetchone()
  cursor.fast_executemany = True
  sql_query="SELECT symbol,exp_date,open,high,low,close,open_int,no_of_trade,bhavcopydate FROM forex where symbol ='%s' and exp_date between '%s' and '%s' and bhavcopydate>'%s'" % (symbol,exp_date1,exp_date2,bhavcopydate)
  try:
    #Execute the SQL command
    cursor.execute(sql_query)
    table_rows = cursor.fetchall()
    df = pd.DataFrame(table_rows,columns=cursor.column_names)
    print(df.head(26))
  except Error as e:
    cprint('-------------------------------','red')
    print("Error while connecting to MySQL", e)
    cprint('-------------------------------','red')
except Error as e:
  cprint('-------------------------------','red')
  print("Error while connecting to MySQL", e)
  cprint('-------------------------------','red')
#simple candle stick plot
# convert into datetime object 
#df['bhavcopydate'] = pd.to_datetime(df['bhavcopydate'])
# apply map function 
#df['bhavcopydate'] = df['bhavcopydate'].map(mpdates.date2num) 
INCREASING_COLOR = 'GREEN'
DECREASING_COLOR = 'RED'

data = [ dict(
    type = 'candlestick',
    open = df['open'],
    high = df['high'],
    low = df['low'],
    close = df['close'],
    x = df['bhavcopydate'],
    yaxis = 'y2',
    name = 'Forex Futures',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()
fig = dict( data=data, layout=layout )
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = True )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='3 mo',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 mo',
            step='month',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

colors = []

for i in range(len(df.close)):
    if i != 0:
        if df.close[i] > df.close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

fig['data'].append( dict( x=df['bhavcopydate'], y=df['no_of_trade'],                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )
fig = go.Figure(data)
fig.show()
#py.iplot( fig, filename = 'candlestick-test-3', validate = False )



Connected to MySQL Server - version 5.7.31
    symbol    exp_date     open  ...  open_int  no_of_trade  bhavcopydate
0   GBPINR  2020-10-27  94.1825  ...     58861        45004    2020-09-28
1   GBPINR  2020-10-27  95.2800  ...     50593        32695    2020-09-29
2   GBPINR  2020-10-27  94.9000  ...     47188        29651    2020-09-30
3   GBPINR  2020-10-27  95.1900  ...     46606        58691    2020-10-01
4   GBPINR  2020-10-27  94.8000  ...     50583        31073    2020-10-05
5   GBPINR  2020-10-27  94.6000  ...     55988        38051    2020-10-06
6   GBPINR  2020-10-27  95.0100  ...     59726        36323    2020-10-07
7   GBPINR  2020-10-27  94.6150  ...     49252        32474    2020-10-08
8   GBPINR  2020-10-27  94.9900  ...     50554        33106    2020-10-09
9   GBPINR  2020-10-27  94.9500  ...     65067        30900    2020-10-12
10  GBPINR  2020-10-27  95.6125  ...     67739        27125    2020-10-13
11  GBPINR  2020-10-27  95.4800  ...     47428        48939    2020-

In [ ]:
#FUTURES ONLY
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2020-04-02','2020-04-06','2020-04-10','2020-04-14','2020-05-01','2020-05-25','2020-10-02','2020-11-16','2020-11-30','2020-12-25']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
      os.mkdir(base+y+'/bhavcopy')
    
    #now lets work with the FUTURES
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        futures_bhavcopy = requests.get('https://archives.nseindia.com/content/historical/DERIVATIVES/'+y+'/'+dmonth[m]+'/fo'+d+dmonth[m]+y+'bhav.csv.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound fo'+d+dmonth[m]+y+'bhav.csv.zip..skipping','red')
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
    if futures_bhavcopy.status_code==200:
        dload=open(zpath, 'wb')
        dload.write(futures_bhavcopy.content)
        dload.close()
        #open the downlaoded bhavcopy and extract it
        z = zipfile.ZipFile(zpath, 'r')
        z.extractall(base+y+'/futures')
        z.close()
        os.remove(zpath)
        #reading and storing in a dictionary
        f, deldict = pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
        #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
        print('fo'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS')
        # making dataframe from csv file
        if os.path.exists(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          df1 = data[data['INSTRUMENT'] == 'FUTIDX'] #dataframe of only FUTIDX rows
          df2 = data[data['INSTRUMENT'] == 'FUTSTK'] #dataframe of only FUTSTK rows
          data = pd.concat([df1,df2]) #concatennating the 2 dataframes
          data=data[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'OPEN', 'HIGH', 'LOW', 'CLOSE','SETTLE_PR','CONTRACTS','VAL_INLAKH','OPEN_INT','CHG_IN_OI']]
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXPIRY_DT'] = pd.to_datetime(data['EXPIRY_DT'], format = '%d-%b-%Y') #the exp_date column is in format dd/mmm/yyyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/futures/'+ str(nextdt.date())+'_futures.csv', index=False) #write the changes above to a new file and add an _futures to the csv file
          os.remove(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv')
          print('Futures bhavcopy '+ str(nextdt.date())+'_futures.csv create...SUCCESS')
                  
          ######################################################################################
          # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
          # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
          # https://pynative.com/python-mysql-database-connection/
          d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
          #check if the path exist and connect to cloud mysql 
          #if os.path.exists(base+y):
          try:
            connection = mysql.connector.connect(host='johnny.heliohost.org',
                                                database='akini_algotrade',
                                                user='akini',
                                                password='Drink7up@home')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server - version", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FUTURES bhavcopy "+'/'+y+'-'+m+'-'+d+'_futures.csv'+" into DB....:", record)
            cursor.fast_executemany = True
            with open(base+y+'/futures/'+y+'-'+m+'-'+d+'_futures.csv', newline='',  encoding="utf8") as csvfile:
              csvdata = csv.reader(csvfile)
              #skip the 1st row as it will be header
              next(csvdata)
              pbar = ProgressBar()
              for row in pbar(list(csvdata)):
                # Prepare SQL query to INSERT a record into the database.
                sql_stocks = "INSERT INTO futures (instrument,symbol, expiry_dt, open, high, low, close, settle_pr,contracts,val_inlakh,open_int,chg_in_oi,bhavcopydate) \
                VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s', '%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                #print(sql)
                try:
                  #Execute the SQL command
                  cursor.execute(sql_stocks)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
          finally:
            if (connection.is_connected()):
              cursor.close()
              connection.close()
if (connection.is_connected()):
  cursor.close()
  connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - FUTURES imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')